### Get Azure Machine Learning Workspace

Workspace is used to retreive credentials and secrets in Key Vault to connect to Azure Data Explorer.

In [ ]:
from obs.drift.drift_analysis_kusto import Drift_Analysis_User
from azureml.core import Workspace
from obs.drift import launch_dashboard, execute

ws = Workspace.from_config()
drift_analysis = Drift_Analysis_User(ws)

### Option 1: Local Dashboard with Dash

Run cell below and copy the link into browser to access the dashboard. Once in the dashboard, interact with the widgets to create visualizations from avaiable data. Use ```irisdata``` created in part 0 as a sample table if needed. 

In [ ]:
launch_dashboard(drift_analysis, port_num=8052)
#change port number from the default 8050 in case you run multiple Dash applications

### Option 2: Launch drift detection job and use  Dashboard to Azure Data Explorer to view result



1. Launch data detection job which is an Azure ML job to compute drift 

In [ ]:
from obs.drift.drift_analysis_scheduler import execute
from datetime import datetime
from azure.ai.ml.constants import TimeZone
from dateutil import tz

from azure.ai.ml.entities import (
    CronSchedule,
    ScheduleStatus,
)

subscription_id= "[YOUR_SUBSCRIPTION]"
resource_group = "[YOUR_RESOURCE_GROUP]"
workspace = "[YOUR_WORKSPACE]"
compute_name = "cpu-cluster"
base_table_name = "IRIS_DATA_NEW"
target_table_name = base_table_name
base_dt_from = "07/19/2022"
base_dt_to = "09/30/2022"
target_dt_from = "10/01/2022" #initial start date for target dataset
target_dt_to ="12/15/2022" #initial endate date for target dataset

cron_schedule = CronSchedule(
    expression="*/10 * * * *", #provide a cron express to schedule the recurring execution of drift detection job
    time_zone=TimeZone.UTC,
    status=ScheduleStatus.ENABLED
)

target_dt_shift_step_size= "W" 
#if this is a scheduled run (target_dt_shift_step_size is provided and cron schedule is provided), 
#the job will take last target_dt_to from result table, use it as new target_dt_from add target_dt_shift_step_size to drive last target_dt_to

ml_client, job_name = execute(subscription_id=subscription_id,resource_group=resource_group,workspace=workspace, compute_name =compute_name, 
base_table_name =base_table_name,target_table_name =target_table_name, base_dt_from =base_dt_from, base_dt_to= base_dt_to,target_dt_from=target_dt_from, 
target_dt_to=target_dt_to, bin="7d", limit=100000, concurrent_run=False, drift_threshold =0.4,target_dt_shift_step_size=target_dt_shift_step_size,
job_cron_schedule=cron_schedule, drift_result_table="data_drift_result")
#remove job_cron_schedule =cron_schedule if you need to test before scheduling the job
#If you need to provide your own data drift logic, please supply the module using user_defined_module_file="drift_job/drift_analysis_user.py", user_defined_conda_file="drift_job/conda.yml", 
# drift_analysis_job_file="drift_job/drift_analysis_aml_job.py",
#check the example of user defined modules in aml-obs-client/obs/test/drift_job


 2.Go to Azure Data Explorer Dashboards and import the quick_start/ADX_dashboards/databoard_drift_detection.json file to create a dashboard.

Review the instructions on how to import the dashboard [here.](https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file)

Default parameters may need to updated to ensure proper connection.